# 活性予測モデルの構築
ここではChMEBLより取得した[DPP4阻害剤のデータ](https://www.ebi.ac.uk/chembl/target_report_card/CHEMBL284/)を利用して活性モデルの作成を行います。

## データのダウンロード

1. [ChEMBLサイト](https://www.ebi.ac.uk/chembl/target_report_card/CHEMBL284/)の中央部の左側のパイチャートのKi(685カウント)をクリックします。
![chembl dpp4](images/chembl_dpp4_1.png)

2. 左のフィルターセクションからBAO labelを選んでsingle proteinのみに絞り込みます。
![chembl dpp4](images/chembl_dpp4_3.png)

3. 続いて、すべての化合物を選択し、TSVでエクスポートします。
![chembl dpp4](images/chembl_dpp4_2.png)

4. データはzip圧縮されてダウンロードするので解答して、わかりやすい名前(dpp4_chembl.tsv)に変更し、このipynbファイルがあるフォルダに移動してください。

## TSVデータの前処理

ダウンロードしたTSVから予測モデルを構築するのに必要な情報のみを抽出する関数を用意します。具体的には数値(standard_relation)が'='であるもの(>=のような曖昧なKiを除く)のみをモデル構築とテストに利用します。また巨大な分子が含まれているので、分子量が6００以上のものは排除しました。

関数としては単純で、タブ区切りのデータをsplitしていきstandard_relationをチェックして上の基準に合致するもののみを出力するようになっています。

In [ ]:
def tsv2valid_tsv(tsvfile, valid_tsvfile):
    with open(tsvfile) as rf:
        with open(valid_tsvfile, "w") as wf:
            header = rf.readline()
            hs = header.split("\t")
            wf.write("{}\t{}\t{}\t{}\n".format(hs[0][1:-1], hs[6][1:-1], hs[7][1:-1], hs[10][1:-1]))
            for l in rf:
                ls = l.split("\t")
                if ls[9] == "\"'='\"" and float(ls[3][1:-1]) < 600:
                    wf.write("{}\t{}\t{}\t{}\n".format(ls[0][1:-1], ls[6][1:-1], ls[7][1:-1], ls[10][1:-1]))

In [ ]:
tsv2valid_tsv("dpp4_chembl.tsv", "dpp4_valid.tsv")

## 予測モデル構築のために利用するライブラリのインポート
- 今回はLightGBMの回帰モデルを利用し、サポートベクター回帰モデルとの比較も行います。
- ハイパーパラメータの最適化にはOptunaを利用します。

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from useful_rdkit_utils import mol2numpy_fp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import optuna
# Logging levelを変えておきます
optuna.logging.set_verbosity(optuna.logging.ERROR)

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
import joblib

import pathlib
import sys
import os
#　実行するノートブックのパスを取得します
notedir = pathlib.Path().resolve()
print(notedir)

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.info')

### 後の描画用にユーティリティ関数を定義しておきます

In [ ]:
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import rdDepictor

def mol2svg(mol):
    rdDepictor.Compute2DCoords(mol)
    d2d = rdMolDraw2D.MolDraw2DSVG(200, 100)
    d2d.DrawMolecule(mol)
    d2d.FinishDrawing()
    return d2d.GetDrawingText()

## DPP４データの読み込み

予測モデルの構築に使うデータを読み込みます。

In [ ]:
df = pd.read_table('./dpp4_valid.tsv', sep='\t')
df.head(5)

In [ ]:
# データの大きさの確認
print(df.shape)

### 塩の取り扱い
データセットに塩を含む分子が含まれているのでモデル構築前に正規化が必要です。01のチュートリアルのコードを利用します。

**ここで復習をしましょう**

In [ ]:
# 塩を含むデータの確認
for smi in df['Smiles']:
    if '.' in smi:
        print(smi)

parent_dir = os.path.abspath(os.path.join(notedir, os.pardir))
# cheminfo_util をimportします。
sys.path.append(parent_dir)
import cheminfo_uti

In [ ]:
# 分子の正規化と合わせてKiの値をpKiに変換します
df['ROMol'] = df['Smiles'].apply(Chem.MolFromSmiles)
df['clean_mol'] = df['ROMol'].apply(cheminfo_util.prep_moleclue) # ここで分子の正規化を実行しています
df['pKi'] = df['Standard Value'].apply(lambda x: 9-np.log10(x))

[10:57:05] Running Uncharger
[10:57:05] Running Uncharger
[10:57:06] Running Normalizer
[10:57:06] Running Uncharger
[10:57:06] Running LargestFragmentChooser
[10:57:06] Fragment: N#C[C@@H]1C[C@@H]2C[C@@H]2N1C(=O)[C@@H](N)C1CCN(S(=O)(=O)Cc2ccccc2)CC1
[10:57:06] New largest fragment: N#C[C@@H]1C[C@@H]2C[C@@H]2N1C(=O)[C@@H](N)C1CCN(S(=O)(=O)Cc2ccccc2)CC1 (54)
[10:57:06] Running Uncharger
[10:57:06] Running Uncharger
[10:57:06] Running Normalizer
[10:57:06] Running Uncharger
[10:57:06] Running LargestFragmentChooser
[10:57:06] Fragment: N#C[C@@H]1C[C@@H]2C[C@@H]2N1C(=O)[C@@H](N)C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1
[10:57:06] New largest fragment: N#C[C@@H]1C[C@@H]2C[C@@H]2N1C(=O)[C@@H](N)C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1 (51)
[10:57:06] Running Uncharger
[10:57:06] Running Uncharger
[10:57:06] Running Normalizer
[10:57:06] Running Uncharger
[10:57:06] Running LargestFragmentChooser
[10:57:06] Fragment: N#C[C@@H]1C[C@@H]2C[C@@H]2N1C(=O)[C@@H](N)C1CCN(C(=O)Cc2ccccc2)CC1
[10:57:06] New largest frag

[10:57:06] Running Uncharger
[10:57:06] Running Uncharger
[10:57:06] Running Normalizer
[10:57:06] Running Uncharger
[10:57:06] Running LargestFragmentChooser
[10:57:06] Fragment: COC[C@H](C)N1Cc2nc(C)c(CN)c(-c3ccc(Cl)cc3Cl)c2C1=O
[10:57:06] New largest fragment: COC[C@H](C)N1Cc2nc(C)c(CN)c(-c3ccc(Cl)cc3Cl)c2C1=O (47)
[10:57:06] Running Uncharger
[10:57:06] Running Uncharger
[10:57:06] Running Normalizer
[10:57:06] Running Uncharger
[10:57:06] Running LargestFragmentChooser
[10:57:06] Fragment: COC[C@H](C)N1Cc2nc(C)c(CN)c(-c3ccc(Cl)cc3Cl)c2C1=O
[10:57:06] New largest fragment: COC[C@H](C)N1Cc2nc(C)c(CN)c(-c3ccc(Cl)cc3Cl)c2C1=O (47)
[10:57:06] Running Uncharger
[10:57:06] Running Uncharger
[10:57:06] Running Normalizer
[10:57:06] Running Uncharger
[10:57:06] Running LargestFragmentChooser
[10:57:06] Fragment: COCCN1Cc2nc(C)c(CN)c(-c3ccc(Cl)cc3Cl)c2C1=O
[10:57:06] New largest fragment: COCCN1Cc2nc(C)c(CN)c(-c3ccc(Cl)cc3Cl)c2C1=O (44)
[10:57:06] Running Uncharger
[10:57:06] Running Unchar

[10:57:11] Running Normalizer
[10:57:11] Running Uncharger
[10:57:11] Running LargestFragmentChooser
[10:57:11] Fragment: CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](Cc1cnc[nH]1)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](C)NC(=O)[C@H](CO)NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](C)NC(=O)[C@H](CCSC)NC(=O)[C@H](CC(=O)O)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](C)NC(=O)[C@@H]1CCCN1C(=O)[C@H](C)NC(=O)[C@H](CC(=O)O)NC(=O)[C@H](CCC(=O)O)NC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CC(N)=O)NC(=O)[C@H](CC(=O)O)NC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCCN)NC(=O)C(NC(=O)[C@@H]1CCCN1C(=O)[C@@H](N)Cc1ccc(O)cc1)C(C)(C)C)C(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](Cc1ccc(O)cc1)C(N)=O)[C@@H](C)O)[C@@H](C)CC
[10:57:11] New largest fragment: CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](Cc1cnc[nH]1)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](C)NC(=O

確認のために最初の５０化合物を表示させてみます。

In [ ]:
Draw.MolsToGridImage(df['clean_mol'][:50], molsPerRow=5)

##　フィンガープリントの生成
mol2numpy_fpは引数にRadiusとBitsを渡せばNumpyのArrayとしてデータを返しますのでこれを利用して入力となるXを作成しましょう。

In [ ]:
# 描画用
clean_mols = df['clean_mol'].to_list()
mols_svgs = [mol2svg(m) for m in clean_mols]
X = np.array([mol2numpy_fp(m, 2, 1024) for m in df['clean_mol']])
y = np.array([float(v) for v in df['pKi']]).ravel()
print(X.shape, y.shape)

### 訓練セット、テストセットの分割
訓練用のデータとテスト用のデータに分割するためにランダムスプリットをおこないます。全データのうち７０%を訓練データに利用し、残りの30%を性能確認のためのテストデータとします。

In [ ]:
train_idx, test_idx = train_test_split([i for i in range(X.shape[0])], train_size=0.7, random_state=111)

In [ ]:
train_X = X[train_idx]
train_svg = [mols_svgs[i] for i in train_idx]
test_X = X[test_idx]
test_svg = [mols_svgs[i] for i in test_idx]
train_y = y[train_idx]
test_y = y[test_idx]

print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

### ハイパーパラメータチューニングを行う
Objective functionの定義
- optunaでハイパーパラメータの最適化を行うためにはobjective関数の定義が必要です。
- 以下のコードでは[cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)で得られるr2の平均値を評価用の値に利用しています。
- チューニングするハイパーパラメータは、そのサンプリングの仕方によって範囲と、[サンプリングメソッド](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial)を変更します。
  - suggest_int 整数をサンプリング
  - suggest_loguniform, 対数一様分布からのサンプリング suggest_float(log=True)が推奨される
  - suggest_uniform　一様分布からのサンプリング suggest_float()が推奨される
  - suggest_categorical　カテゴリ変数からのサンプリング
  - etc.

In [ ]:
def objective(trial, x, t, cv):
    # 1. 最適化するパラメータを設定します
    # https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html
    n_estimaters = trial.suggest_int('n_estimators', 1, 100) #Boost Treeの数
    max_depth = trial.suggest_int('max_depth', 1, 10) #探索木の深さ
    num_leaves = trial.suggest_int('num_leaves', 2, 10)
    min_child_weight = trial.suggest_float("min_child_weight", 0.1, 10, log=True)
    subsample = trial.suggest_float("subsample",0.55, 0.95)    
    colsample_bytree = trial.suggest_float("subsample",0.55, 0.95)    

    # 2. LightGBMREgressor
    estimator = LGBMRegressor(
        n_estimators=n_estimaters,
        max_depth=max_depth,
        num_leaves=num_leaves,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree= colsample_bytree,
        random_state=111,
        verbose=-1
    )

    # 3. 学習の実行、検証結果の表示
    print('Current_params : ', trial.params)
    r2 = cross_val_score(estimator, x, t, cv=cv, scoring="r2").mean()
    print(r2)
    print("#######")
    return r2

In [ ]:
# r2の最適化なので方向性は最大となるように設定します
study = optuna.create_study(direction='maximize')
cv = 10

In [ ]:
# n_trials 50だと時間が少しかかるかもしれません。実行したら、しばし休憩しましょう。
study.optimize(lambda trial: objective(trial, train_X, train_y, cv), n_trials=50)

### 最適化後のR2値は0．6前後でした。実際にテストデータをプロットして確認してみましょう

In [ ]:
print(study.best_value)

Optunaの結果得られた最良のハイパーパラメータを用いて予測モデルを構築します。

In [ ]:
best_lgbm = LGBMRegressor(**study.best_params)
best_lgbm.fit(train_X, train_y)

In [ ]:
pred_y = best_lgbm.predict(test_X)
pred_train_y = best_lgbm.predict(train_X)

## モデルの性能の視覚化
予測モデルの性能を確認するため、「訓練データの予測結果」「テストセットの予測結果」を実測値に比べてどのくらいズレているかをプロットします。

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
plt.title('LightGBM model for DPP4 activity prediction')
plt.style.use('ggplot')
plt.scatter(pred_train_y, train_y, alpha=0.8, c='pink')
plt.scatter(pred_y, test_y, alpha=0.4, c='blue')
plt.plot(np.linspace(4,9.5), np.linspace(4,9.5))
plt.xlabel('predicted pKi')
plt.ylabel('acctual pKi')
plt.show()

## より高度な視覚化
上記のプロットではどの点がどの化合物なのか分かりづらいため、各点をマウスオーバーしたときに構造が表示されるようにします。

In [ ]:
from bokeh.io import push_notebook, show, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.plotting import ColumnDataSource
from bokeh.transform import factor_cmap
output_notebook()

In [ ]:
line_data =  dict(x=np.linspace(4,9.5), y=np.linspace(4,9.5))
train_data = dict(x=pred_train_y,
       y=train_y,
        ids = train_idx,
        img = train_svg,
        #label=[str(l) for l in hdbscan.labels_]
        )
test_data = dict(x=pred_y,
       y=test_y,
        ids = test_idx,
        img = test_svg,
        #label=[str(l) for l in hdbscan.labels_]
        )

TOOLTIPS = """
<div>
index: @ids<br>
<div>@img{safe}</div>
</div>
"""
lince_source = ColumnDataSource(line_data)
train_source = ColumnDataSource(train_data)
test_source =  ColumnDataSource(test_data)
p = figure(tooltips=TOOLTIPS, width=500, height=500,)
p.title = 'LightGBM model for DPP4 activity prediction'
p.xaxis.axis_label = 'predicted pKi'
p.yaxis.axis_label = 'expelimental pKi'
p.line(source=line_data)
c = p.circle('x', 'y', size=10, source=train_source, 
         fill_color='skyblue',
         alpha=0.8
        )
p.circle('x', 'y', size=10, source=test_source, 
         fill_color='pink',
         alpha=0.8
        )
handle=show(p, notebook_handle=True)
push_notebook(handle=handle)

## サポートベクターマシンでの予測モデル
同様にSVRによる予測モデルも構築します。コードの流れは同じなので説明は省略します。

In [ ]:
def svm_objective(trial, x, t, cv):
    # 1. 最適化するパラメータを設定します
    kernel = trial.suggest_categorical('kernel', [#'linear',
                                                  #'poly', 
                                                  'rbf', 
                                                  #'sigmoid'
                                       ]) #Kernelの種類
    C = trial.suggest_float('C', 0.01, 1000, log=True) #C
    epsilon = trial.suggest_float('epsilon', 0.001, 10, log=True)
    

    # 2. LightGBMREgressor
    estimator = SVR(
        kernel=kernel,
        C=C,
        epsilon=epsilon,
    )

    # 3. 学習の実行、検証結果の表示
    print('Current_params : ', trial.params)
    r2 = cross_val_score(estimator, x, t, cv=cv, scoring="r2").mean()
    print(r2)
    print("#######")
    return r2

In [ ]:
svm_study = optuna.create_study(direction='maximize')
cv = 10
svm_study.optimize(lambda trial: svm_objective(trial, train_X, train_y, cv), n_trials=50)

In [ ]:
svm_study.best_value

In [ ]:
best_svm = SVR(**svm_study.best_params)
best_svm.fit(train_X, train_y)

In [ ]:
pred_y = best_svm.predict(test_X)
pred_train_y = best_svm.predict(train_X)
plt.clf()
plt.title('SVM model for DPP4 activity prediction')
plt.style.use('ggplot')
plt.scatter(pred_train_y, train_y, alpha=0.8, c='pink')
plt.scatter(pred_y, test_y, alpha=0.4, c='blue')
plt.plot(np.linspace(4,9.5), np.linspace(4,9.5))
plt.xlabel('predicted pKi')
plt.ylabel('acctual pKi')
plt.show()

In [ ]:
line_data =  dict(x=np.linspace(3.5,10), y=np.linspace(3.5,10))
train_data = dict(x=pred_train_y,
       y=train_y,
        ids = train_idx,
        img = train_svg,
        #label=[str(l) for l in hdbscan.labels_]
        )
test_data = dict(x=pred_y,
       y=test_y,
        ids = test_idx,
        img = test_svg,
        #label=[str(l) for l in hdbscan.labels_]
        )

TOOLTIPS = """
<div>
index: @ids<br>
<div>@img{safe}</div>
</div>
"""
lince_source = ColumnDataSource(line_data)
train_source = ColumnDataSource(train_data)
test_source =  ColumnDataSource(test_data)
p = figure(tooltips=TOOLTIPS, width=500, height=500,)
p.title = 'SVM model for DPP4 activity prediction'
p.xaxis.axis_label = 'predicted pKi'
p.yaxis.axis_label = 'expelimental pKi'
p.line(source=line_data)
c = p.circle('x', 'y', size=10, source=train_source, 
         fill_color='skyblue',
         alpha=0.8
        )
p.circle('x', 'y', size=10, source=test_source, 
         fill_color='pink',
         alpha=0.8
        )
handle=show(p, notebook_handle=True)
push_notebook(handle=handle)

### 考察
今回の例ではLGBM, SVMあまりパフォーマンスに差がありませんでした。SVMはそのアルゴリズムからデータが増えると遅くなるので、この程度の差であればLGBMでも良いかもしれません。
なお、SVMは過学習しているように見えます。

### 予測モデルで自身のアイディア化合物の予測
以下の例ではSMILESの入力を受取り、予測値を返す関数を定義しています。

In [ ]:
def predict_dpp4act(smiles, model=best_lgbm):
    mol = Chem.MolFromSmiles(smiles)
    fp = mol2numpy_fp(mol, 2, 1024)
    val = model.predict([fp])
    return val

In [ ]:
smi = 'Fc1cc(c(F)cc1F)C[C@@H](N)CC(=O)N3Cc2nnc(n2CC3)C(F)(F)F'
mol = Chem.MolFromSmiles(smi)
mol

In [ ]:
predict_dpp4act(smi)

In [ ]:
smi = 'Fc1cc(c(F)cc1F)C[C@@H](N)CC(=O)N3Cc2nnc(n2CC3)C(C)(C)C'
mol = Chem.MolFromSmiles(smi)
mol

In [ ]:
predict_dpp4act(smi)

## 作成したモデルの保存
joblibを利用すれば、構築したモデルを簡単に保存、呼び出せます。

In [ ]:
joblib.dump(best_lgbm, 'dpp4_lgbm.pkl2', compress=3)

In [ ]:
model = joblib.load('./dpp4_lgbm.pkl2')

In [ ]:
res = model.predict(train_X)

In [ ]:
res[:10]

# 演習
ChEMBLからデータの取得、予測モデル構築、性能評価をDPP4ではなく別のターゲットで実施してください。
例) [JAK](https://www.ebi.ac.uk/chembl/target_report_card/CHEMBL2835/)

思いつかない場合にはスタッフに聞いてみてください。

# 補足

## RDBを利用したデータの取得
もし、自分のラップトップやサーバーにChEMBLのデータを入れてある場合、SQLでデータを取得することができます。セキュリティの面からもこちらの方が望ましいです。postgreSQLを利用する場合のコードを載せておきます。

## 実行したSQLこー

```sql
SELECT m.chembl_id AS compound_chembl_id,   
s.canonical_smiles,   
r.compound_key,   
COALESCE(TO_CHAR(d.pubmed_id, 'FM99999999'),d.doi) AS pubmed_id_or_doi,   
a.description                   AS assay_description,   act.standard_type,   
act.standard_relation,   
act.standard_value,   
act.standard_units,   
act.activity_comment,
a.assay_category 
FROM compound_structures s
 RIGHT JOIN molecule_dictionary m on s.molregno = m.molregno 
 JOIN compound_records r on m.molregno = r.molregno  
 JOIN docs d on r.doc_id = d.doc_id 
 JOIN activities act on r.record_id = act.record_id
 JOIN assays a on act.assay_id = a.assay_id 
 JOIN target_dictionary t on a.tid = t.tid 
AND t.chembl_id = 'CHEMBL284'
AND act.standard_type = 'Ki'
AND act.standard_relation = '='
AND a.confidence_score > 8
;
```

上記のSQLをgetdata.sqlというファイルで保存して以下のコマンドを実行すればtsvファイルとしてDPP4のデータを取得できます。

```
psql -d chembl_33 -f getdata.sql -A -F $'\t' > ddp4_dataset.tsv
```

## Discussion（ちょっと意見交換）
- ChEMBLから今回のようなデータセットを作成する場合、皆様ならどのような基準でデータをまとめますか？
- doc_id毎？　IC50, Ki?　confidence_score?